In [2]:
# -*- coding: utf-8 -*-
"""
Created on Thu May 12 23:24:54 2016

@author: stuka

"""

import numpy as np
import nltk
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
import os
import io
import re
from collections import defaultdict
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import math
import pandas as pd
import sys
import gensim
from string import punctuation
import itertools
from random import shuffle
from nltk.corpus import stopwords
import random
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.cluster.vq import kmeans,vq

#Cleaning the house
os.getcwd()
os.chdir("/home/christian/Documents/ITAM_CDA/Mineria de texto/final_project/mineria-texto-diario-debates/code/")
os.listdir(".")

path_to_raw = "/home/christian/Documents/ITAM_CDA/Mineria de texto/final_project/mineria-texto-diario-debates/data/"


# Cambiar representación binaria de los archivos

In [3]:
file_names = [f for f in os.listdir(path_to_raw+'raw/') if f.endswith('.txt')]

In [ ]:
documentos =[io.open(f,'rt',encoding='ISO-8859-1') for f in file_names]


for file_name in file_names:
    sourceEncoding = "iso-8859-1"
    targetEncoding = "utf-8"
    source = io.open(file_name,'rt',encoding='ISO-8859-1')
    target = open(path_to_raw+'encoded/'+file_name, "w")
    target.write(source.read().encode(targetEncoding))
    

In [4]:
documentos =[io.open(path_to_raw+'encoded/'+f,'rt') for f in file_names]

In [5]:
#Pruebas
#raw = documentos[0].read()

documentos[1]

#Codigo para probar stopterm
for documento in documentos:
    print(documento.name.split('/')[len(documento.name.split('/'))-1],len(documento.read().replace("Honorable Asamblea:","Honorable asamblea:").split("Honorable asamblea:")))
    

('dd_1958.txt', 309)
('dd_1946.txt', 370)
('dd_1970.txt', 444)
('dd_1964.txt', 736)
('dd_1976.txt', 598)
('dd_1952.txt', 428)


Algunas funciones de limpieza de texto

In [6]:
def toLowerCase(s):
    data = s.lower()
    return data

def mataAcentos(s):
    charstosub = pd.DataFrame(zip([u'á', u'é', u'í', u'ó', u'ú',u'"',u'“',u'”',u',',u'\.',u'ñ',u'\!',u'\¡'],[u'a', u'e', u'i', u'o', u'u',u'',u'',u'',u'',u'',u'n',u'\.',u'\.'])) 
    data = s.lower()
    for row in charstosub.iterrows():
        data = re.sub(row[1][0],row[1][1],data)
    return data


#Esto si funciona
def strip_punctuation(s):
    return ''.join(c for c in s if c not in punctuation)

Se crean los documentos tematicas eliminando puntuación y acentos para el algoritmo Doc2Vec

In [7]:
#Este es el buen codigo
"""
documento trae un file abierto
lo consumo a memoria con read() a un solo string
lo parto segun keyterms = Honorable asamblea
"""
for documento in documentos:
    documento_nombre = documento.name.rsplit('/',1)[1].split('.')[0]
    raw = documento.read()
    for i,tematica in enumerate(raw.replace("Honorable Asamblea:","Honorable asamblea:").split("Honorable asamblea:")):
        with io.open(path_to_raw+'tematicas/'+documento_nombre+'_'+str(i+1)+'.txt', mode="w") as newfile:
            tematica_limpia = mataAcentos(strip_punctuation(tematica.replace('\n',' ')))
            newfile.write(tematica_limpia)
            

TypeError: must be unicode, not str

Se crean los documentos tematicas sin tocar puntuación y acentos para el algoritmo RAKE

In [ ]:
for documento in documentos:
    documento_nombre = documento.name.rsplit('/',1)[1].split('.')[0]
    raw = documento.read()
    for i,tematica in enumerate(raw.replace("Honorable Asamblea:","Honorable asamblea:").split("Honorable asamblea:")):
        with io.open(path_to_raw+'tematicas_to_rake/'+documento_nombre+'_'+str(i+1)+'.txt', mode="w") as newfile:
            tematica_limpia = toLowerCase(tematica)
            newfile.write(tematica_limpia)

# Doc2Vec

In [8]:

file_names_tematicas = [f for f in os.listdir(path_to_raw+'tematicas/') if f.endswith('.txt')]
sentences = []
i=0
dicto = {}
for doc in file_names_tematicas:
    f = io.open(path_to_raw+'tematicas/'+doc,'rt')
    sentences.append(gensim.models.doc2vec.LabeledSentence(f.read().split(),[doc]))
    f.close()
    
    
#Pruebas
sentences[0]
sentences[0:5]
print(sentences[0:5])
len(sentences)


[TaggedDocument(words=[u'a', u'la', u'segunda', u'comision', u'de', u'hacienda', u'se', u'turno', u'la', u'solicitud', u'de', u'jubilacion', u'voluntaria', u'por', u'mas', u'de', u'30', u'anos', u'de', u'servicios', u'presentada', u'por', u'el', u'c', u'miguel', u'angel', u'beltran', u'calderon', u'jefe', u'de', u'departamento', u'de', u'glosa', u'de', u'la', u'contaduria', u'mayor', u'de', u'hacienda', u'el', u'peticionario', u'se', u'funda', u'en', u'lo', u'que', u'dispone', u'la', u'fraccion', u'iii', u'del', u'articulo', u'3\xba', u'de', u'la', u'ley', u'de', u'jubilaciones', u'para', u'funcionarios', u'y', u'empleados', u'del', u'poder', u'legislativo', u'al', u'estudiar', u'los', u'documentos', u'exhibidos', u'por', u'el', u'interesado', u'se', u'llega', u'a', u'la', u'siguiente', u'conclusion', u'a', u'que', u'presto', u'servicios', u'en', u'la', u'secretaria', u'de', u'industria', u'y', u'comercio', u'del', u'5', u'de', u'agosto', u'al', u'10', u'de', u'diciembre', u'de', u'193

2885

In [9]:
#Modelo
min_count = 5 # Ignora todo documento que tenga cinco palabras o menos
size = 300 # Numero de dimensiones (tanto de las palabras como de los documentos)
window = 10 # Cuantas palabras hacia adelante / atrás usa para trabajar
 
model = gensim.models.doc2vec.Doc2Vec(sentences,size = size, window = window, min_count = min_count)


In [10]:
dir(model)
len(model.vocab)
model.similarity('pri','pan')

0.80203930688603942

In [11]:
def checa(word):
    for i in model.most_similar(word):
        print(i)
        
checa(u'fraudes')

(u'procesos', 0.7559369802474976)
(u'padrones', 0.6903518438339233)
(u'hogares', 0.6877034902572632)
(u'escrutinios', 0.6736383438110352)
(u'latifundistas', 0.655331552028656)
(u'movimientos', 0.6515790224075317)
(u'anteproyectos', 0.6494857668876648)
(u'campesinos', 0.647720456123352)
(u'sindicatos', 0.6443589925765991)
(u'comicios', 0.6443477869033813)


In [147]:
#Pruebas para similitud de documentos
#print model.docvecs.most_similar(["dd_1970_8.txt"])
#model.docvecs["dd_1970_8.txt"]
#dir(model.docvecs)
#model.docvecs.doctags
#dir(model)

Se prepara la matriz de representación vectorial de documentos y los nombres de estos

In [12]:
mylist = []
tags=[]
for doc in model.docvecs.doctags:
    tags.append(doc)    
    mylist.append(model.docvecs[doc])

model.docvecs.doctags
tags[0]
mat = np.array(mylist)

### K means

In [13]:
for i in range(10):
    agrupamiento = KMeans(n_clusters = i+1)
    agrupamiento.fit(mat)
#    mylist['clasificacion_'] = agrupamiento.labels_.tolist()

In [150]:
from scipy.cluster.vq import kmeans,vq
K = range(1,10)

KM = [kmeans(mat,k) for k in K]
centroids = [cent for (cent,var) in KM] 
avgWithinSS = [var for (cent,var) in KM] # suma de cuadrados promedio intra-cluster 

In [151]:
##### plot ###
kIdx = 12
kIdx2 = 4
kIdx3 = 17

# elbow curve
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(K, avgWithinSS, 'b*-')
ax.plot(K[kIdx], avgWithinSS[kIdx], marker='o', markersize=12, 
    markeredgewidth=2, markeredgecolor='r', markerfacecolor='None')
ax.plot(K[kIdx2], avgWithinSS[kIdx2], marker='o', markersize=12, 
    markeredgewidth=2, markeredgecolor='r', markerfacecolor='None')
ax.plot(K[kIdx3], avgWithinSS[kIdx3], marker='o', markersize=12, 
    markeredgewidth=2, markeredgecolor='r', markerfacecolor='None')
plt.grid(True)
plt.xlabel('Numero de clusters')
plt.ylabel('Suma de cuadrados promedio intra-cluster')
plt.title('Criterio del codo para K-Medias')

plt.show()

In [14]:
n_clusters=5

In [15]:
# KMeans con el resultado del criterio del codo
agrupamiento = KMeans(n_clusters = n_clusters)
agrupamiento.fit(mat)
agrupamiento.labels_

array([3, 1, 3, ..., 1, 1, 2], dtype=int32)

In [16]:
agrupados = pd.DataFrame(zip(tags, agrupamiento.labels_.tolist()))

lista_cluster =  [[] for i in range(n_clusters)]
for i in range(len(agrupados)):
    lista_cluster[agrupados[1][i]].append(agrupados[0][i])

In [17]:
for i in range(len(lista_cluster)):
    print("Cluster"+str(i),len(lista_cluster[i]))

('Cluster0', 196)
('Cluster1', 743)
('Cluster2', 372)
('Cluster3', 1335)
('Cluster4', 239)


Debido al costo computacional de usar RAKE sobre archivos grandes es necesario realizar una muestra aleatoria de los documentos de cada cluster

In [ ]:

sample_docs = 40

Se concatenan todos los archivos de un mismo cluster en un solo archivo de entrada al algoritmo RAKE, se toma la versión que tiene acentos y puntuación

In [197]:
for i in range(len(lista_cluster)):
    with io.open(path_to_raw+'clusters_to_rake/'+"cluster_"+str(i)+".txt", 'w') as outfile:
        filenames = lista_cluster[i]
        if len(filenames)<40:
            randfiles = random.sample(filenames, len(filenames))
        else:
            randfiles = random.sample(filenames, 40)
        for fname in randfiles:
            with io.open(path_to_raw+'tematicas_to_rake/'+fname) as infile:
                for line in infile:
                    outfile.write(line)

['dd_1970_328.txt',
 'dd_1964_142.txt',
 'dd_1970_202.txt',
 'dd_1976_237.txt',
 'dd_1970_320.txt',
 'dd_1976_391.txt',
 'dd_1964_149.txt',
 'dd_1958_133.txt',
 'dd_1970_311.txt',
 'dd_1976_408.txt',
 'dd_1952_99.txt',
 'dd_1976_323.txt',
 'dd_1964_260.txt',
 'dd_1976_291.txt',
 'dd_1964_263.txt',
 'dd_1964_221.txt',
 'dd_1970_281.txt',
 'dd_1946_10.txt',
 'dd_1976_260.txt',
 'dd_1970_208.txt',
 'dd_1958_140.txt',
 'dd_1970_254.txt',
 'dd_1952_102.txt',
 'dd_1952_141.txt',
 'dd_1970_245.txt',
 'dd_1970_238.txt',
 'dd_1964_272.txt',
 'dd_1970_239.txt',
 'dd_1946_89.txt',
 'dd_1976_417.txt',
 'dd_1952_106.txt',
 'dd_1946_102.txt',
 'dd_1970_154.txt',
 'dd_1946_31.txt',
 'dd_1964_222.txt',
 'dd_1964_144.txt',
 'dd_1958_55.txt',
 'dd_1946_19.txt',
 'dd_1976_422.txt',
 'dd_1976_272.txt',
 'dd_1946_150.txt',
 'dd_1946_133.txt',
 'dd_1964_238.txt',
 'dd_1946_42.txt',
 'dd_1952_120.txt',
 'dd_1958_144.txt',
 'dd_1964_193.txt',
 'dd_1964_127.txt',
 'dd_1964_115.txt',
 'dd_1964_172.txt',
 'dd_19

In [18]:
for i in range(len(lista_cluster)):
    with io.open(path_to_raw+'clusters_to_rake/'+"cluster_"+str(i)+".txt", 'w') as outfile:
        filenames = lista_cluster[i]
#        if len(filenames)<40:
#            randfiles = random.sample(filenames, len(filenames))
#        else:
#            randfiles = random.sample(filenames, 40)
#        for fname in randfiles:
        for fname in filenames:
            with io.open(path_to_raw+'tematicas_to_rake/'+fname) as infile:
                for line in infile:
                    outfile.write(line)

# MajorClust

Se utiliza la metrica de similitud de cosenos como función de distancia para el algoritmo. Recordar que entre más cerca de 1 se este más similitud entre los documentos. 

In [ ]:
#### Calculo de Cosine Similarity
num_of_samples, num_of_features = mat.shape

magnitudes = np.zeros((num_of_samples))
# this loop can be removed?
for doc_id in range(num_of_samples):
    magnitudes[doc_id] = np.sqrt(mat[doc_id].dot(mat[doc_id].T).sum())

cosine_distances = np.zeros((num_of_samples, num_of_samples))
# this loop can be improved
for doc_id, other_id in itertools.combinations(range(num_of_samples), 2):
    distance = (mat[doc_id].dot(mat[other_id].T).sum())/(magnitudes[doc_id]*magnitudes[other_id])
    cosine_distances[doc_id, other_id] = cosine_distances[other_id, doc_id] = distance

Implememntación propia del Algortmo de Major Clust
Adaptada de http://baali.muse-amuse.in/posts/clustering-text-using-majorclust.html

In [ ]:
##### MajorClust Implementation 
finish = False
iters = 1
shuffled_indices = np.arange(num_of_samples)
shuffle(shuffled_indices)
stitch = shuffled_indices.copy()

while not finish:
    finish = True
    for index in shuffled_indices:
    # aggregating edge weights  
    # bincount cuanta el numero de ocurrencias de un valor en un arreglo y lo pesa por weights
        new_index = np.argmax(np.bincount(shuffled_indices,weights=cosine_distances[index]))
        if shuffled_indices[new_index] != shuffled_indices[index]:
            shuffled_indices[index] = shuffled_indices[new_index]
            finish = False

In [ ]:
clusters = pd.DataFrame(zip(stitch,shuffled_indices),columns=["document","cluster"]).sort_values("document",ascending=True)

In [ ]:
#Cuantos documentos por cluster hubieron?
my_tab = pd.crosstab(index=clusters["cluster"],columns="count").sort_values("count",ascending=False)  

In [ ]:
#Se construyen las estructuras para enviar los documentos de cada cluster a RAKE
rename = pd.DataFrame(zip(my_tab.index,np.arange(my_tab.shape[0])),columns=["cluster","cluster_id"])

final_majorclust=pd.merge(clusters,rename,how='left',on='cluster')
#Conservamos unicamente cierta cantidad de cluster, muchos solo tienen un documento por lo que no los vamos a conservar
nclust = 5
final_clusters = final_majorclust[final_majorclust["cluster_id"]<nclust]
doc_names = [tags[i] for i in final_clusters["document"]]
final_clusters["doc_names"] = doc_names

In [ ]:
agrupados = final_clusters[["doc_names","cluster_id"]]
n_clusters=len(agrupados["cluster_id"].unique())
lista_cluster =  [[] for i in range(n_clusters)]
for i in range(len(agrupados)):
    lista_cluster[agrupados.iloc[i,1]].append(agrupados.iloc[i,0])

for i in range(len(lista_cluster)):
    print("Clster"+str(i),len(lista_cluster[i]))

In [ ]:
sample_docs = 40

In [ ]:
for i in range(len(lista_cluster)):
    with io.open(path_to_raw+'clusters_to_rake/'+"cluster_"+str(i)+".txt", 'w') as outfile:
        filenames = lista_cluster[i]
        if len(filenames)<40:
            randfiles = random.sample(filenames, len(filenames))
        else:
            randfiles = random.sample(filenames, 40)
        for fname in randfiles:
            with io.open(path_to_raw+'tematicas_to_rake/'+fname) as infile:
                for line in infile:
                    outfile.write(line)

### RAKE

In [19]:
import rake
import operator

In [216]:
! pwd

/home/christian/Documents/ITAM_CDA/Mineria de texto/final_project/mineria-texto-diario-debates/code


In [158]:
#nltk.download()
nltk.corpus.stopwords.words('spanish')

[u'de',
 u'la',
 u'que',
 u'el',
 u'en',
 u'y',
 u'a',
 u'los',
 u'del',
 u'se',
 u'las',
 u'por',
 u'un',
 u'para',
 u'con',
 u'no',
 u'una',
 u'su',
 u'al',
 u'lo',
 u'como',
 u'm\xe1s',
 u'pero',
 u'sus',
 u'le',
 u'ya',
 u'o',
 u'este',
 u's\xed',
 u'porque',
 u'esta',
 u'entre',
 u'cuando',
 u'muy',
 u'sin',
 u'sobre',
 u'tambi\xe9n',
 u'me',
 u'hasta',
 u'hay',
 u'donde',
 u'quien',
 u'desde',
 u'todo',
 u'nos',
 u'durante',
 u'todos',
 u'uno',
 u'les',
 u'ni',
 u'contra',
 u'otros',
 u'ese',
 u'eso',
 u'ante',
 u'ellos',
 u'e',
 u'esto',
 u'm\xed',
 u'antes',
 u'algunos',
 u'qu\xe9',
 u'unos',
 u'yo',
 u'otro',
 u'otras',
 u'otra',
 u'\xe9l',
 u'tanto',
 u'esa',
 u'estos',
 u'mucho',
 u'quienes',
 u'nada',
 u'muchos',
 u'cual',
 u'poco',
 u'ella',
 u'estar',
 u'estas',
 u'algunas',
 u'algo',
 u'nosotros',
 u'mi',
 u'mis',
 u't\xfa',
 u'te',
 u'ti',
 u'tu',
 u'tus',
 u'ellas',
 u'nosotras',
 u'vosostros',
 u'vosostras',
 u'os',
 u'm\xedo',
 u'm\xeda',
 u'm\xedos',
 u'm\xedas',
 u

In [20]:
rake_object = rake.Rake("stop_words_spanish.txt", 5, 5, 8)


In [226]:
#Pruebas
sample_file = open(path_to_raw+'clusters_to_rake/'+"cluster_0.txt", 'r')
#sample_file = open(path_to_raw+'tematicas_to_rake/'+"dd_1952_12.txt", 'r')
text = sample_file.read()
keywords = rake_object.run(text)
print "Keywords:", keywords

Keywords: [('son leg\xc3\xadtimas y v\xc3\xa1lidas', 26.652636157192433), ('c\xc3\xa1mara de diputados del h', 26.117857273152683), ('primera comisi\xc3\xb3n revisora de credenciales', 24.44771269298402), ('son diputados de mayor\xc3\xada a', 24.23788446445316), ('quincuag\xc3\xa9sima legislatura del h', 22.53728066122021), ('votaci\xc3\xb3n econ\xc3\xb3mica se pregunta', 22.0475650420007), ('comisi\xc3\xb3n revisora de credenciales', 20.54045462846789), ('constituci\xc3\xb3n pol\xc3\xadtica de', 20.169754285297593), ('julio del presente a\xc3\xb1o', 19.998009888616068), ('son diputados al congreso de', 18.568596403112533), ('c\xc3\xa1mara de diputados', 16.986925590551657), ('son diputados propietario y suplente', 16.777962342724194), ('elecciones a diputados federales celebradas', 16.152053604870094), ('partido aut\xc3\xa9ntico de', 15.530283916630449), ('partido acci\xc3\xb3n nacional', 15.497404096366418), ('comisi\xc3\xb3n somete a', 15.365036159187616), ('primera comisi\xc3\xb3n 

In [249]:
sample_file1 = open(path_to_raw+'clusters_to_rake/'+"cluster_1.txt", 'r')
text1 = sample_file1.read()
keywords1 = rake_object.run(text1)
print "Keywords:", keywords1

Keywords: [('hacienda y cr\xc3\xa9dito p\xc3\xbablico', 26.96456292062732), ('c\xc3\xa1mara de diputados del h', 23.790774278766172), ('secretar\xc3\xada de hacienda y', 19.869820847776968), ('constituci\xc3\xb3n pol\xc3\xadtica de', 19.69126606809537), ('secretar\xc3\xada de gobernaci\xc3\xb3n', 19.52024994817861), ('sesi\xc3\xb3n y se cita', 19.040634968821543), ('secretar\xc3\xada de hacienda', 15.9702083928792), ('orden del d\xc3\xada', 15.887771907754393), ('votaci\xc3\xb3n econ\xc3\xb3mica', 15.62382315923373), ('que se refiere a', 15.310921380321435), ('c\xc3\xa1mara de diputados', 15.008434078550248), ('secretario altamirano mar\xc3\xadn', 15.00382426452266), ('c\xc3\xa1mara de senadores', 14.982508445685227), ('est\xc3\xa1 a discusi\xc3\xb3n', 14.938170945756815), ('secretario guerrero esquivel fernando', 14.830846210444783), ('se procede a recoger', 14.683731982828292), ('departamento del distrito federal', 14.654041157859226), ('banco de m\xc3\xa9xico', 14.608144578512546), 

In [256]:
sample_file2 = open(path_to_raw+'clusters_to_rake/'+"cluster_2.txt", 'r')
#sample_file = open(path_to_raw+'tematicas_to_rake/'+"dd_1952_12.txt", 'r')
text2 = sample_file2.read()
keywords2 = rake_object.run(text2)
print "Keywords:", keywords2

Keywords: [('c\xc3\xa1mara de diputados del h', 23.135610888675192), ('votaci\xc3\xb3n econ\xc3\xb3mica se pregunta', 22.909033135962733), ('se\xc3\xb1oras y se\xc3\xb1ores', 19.16956425095261), ('constituci\xc3\xb3n pol\xc3\xadtica de', 18.79631394458803), ('secretar\xc3\xada de gobernaci\xc3\xb3n', 18.58045937870785), ('a trav\xc3\xa9s de', 16.048985114622084), ('votaci\xc3\xb3n econ\xc3\xb3mica', 15.940887560703798), ('reyes jos\xc3\xa9 mar\xc3\xada', 15.628892396439166), ('partido aut\xc3\xa9ntico de', 15.333312230391124), ('orden del d\xc3\xada', 15.30482943876068), ('partido acci\xc3\xb3n nacional', 14.94064345834785), ('est\xc3\xa1 a discusi\xc3\xb3n', 14.564829683096068), ('c\xc3\xa1mara de diputados', 14.486617335108031), ('pueblo de m\xc3\xa9xico', 14.3566014271252), ('c\xc3\xa1mara de senadores', 14.263742717673187), ('primera comisi\xc3\xb3n dictaminadora', 14.256214861420148), ('ciudad de m\xc3\xa9xico', 14.214800622887093), ('sala de comisiones de', 14.052804479755233), (

In [21]:
file_names_clusters = [f for f in os.listdir(path_to_raw+'clusters_to_rake/') if f.endswith('.txt')]
keywords_len = []
for doc in file_names_clusters:
    documento_nombre = doc.split('.')[0]
    sample_file = io.open(path_to_raw+'clusters_to_rake/'+doc, 'r')
    text = sample_file.read()
    keywords = rake_object.run(text)
    keywords_len = len(keywords)
    with io.open(path_to_raw+'resultados_rake/'+documento_nombre+'_keywords'+'.csv', mode="w",encoding="utf-8") as newfile:
        for i,j in keywords:
            line = i+","+str(j)+"\n"
            newfile.writelines(line)

# TF-IDF
Aplicar TF-IDF a los terminos clave encontrados por RAKE para resaltar aquellos que diferencian realmente a los documentos de un cluster respecto a los demas cluster

In [22]:
########################################### TF IDF #####################################################
#Se preparan los terminos clave de cada cluster para el algoritmo de TF - IDF
# Cada grupo de palabras clave debe quedar en una lista por separado concatenada por espacios
file_names_resultados = [f for f in os.listdir(path_to_raw+'resultados_rake/') if f.endswith('.csv')]
oraciones = [[] for i in range(len(file_names_resultados))]
for doc in file_names_resultados:
    orden = int(doc.split("_")[1])
    with io.open(path_to_raw+'resultados_rake/'+doc, 'rb') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        oracion = ""
        for row in reader:
            oracion = oracion+" "+row[0]
        oraciones[orden]=oracion
        

In [23]:
#Algoritmo TF-IDF
#Se especifican los n-gramas para que se puedan encontrar terminos multipalabra
vectorizer = TfidfVectorizer(ngram_range=(1,3),stop_words = stopwords.words("spanish"))
X = vectorizer.fit_transform(oraciones)

In [27]:
palabras = vectorizer.get_feature_names()
dense = X.todense()
columns = ["cluster","rango","termino","score"]
df = pd.DataFrame(columns=columns)
for j in range(len(oraciones)):
    docto = dense[j].tolist()[0]
    phrase_scores = [pair for pair in zip(range(0, len(docto)), docto) if pair[1] > 0]
    sorted_phrase_scores = sorted(phrase_scores, key=lambda t: t[1] * -1)
    i = 0
    for phrase, score in [(palabras[word_id], score) for (word_id, score) in sorted_phrase_scores][:30]:
        df.loc[j*30+i] = [j,i,phrase,score]       
        i=i+1
df

,cluster,rango,termino,score
0,0,0,comisión,0.175791
1,0,1,artículo,0.137337
2,0,2,secretario,0.131843
3,0,3,josé,0.104376
4,0,4,gobierno,0.090642
5,0,5,legislatura,0.090642
6,0,6,licenciado,0.090642
7,0,7,república,0.085149
8,0,8,permiso,0.081188
9,0,9,fojas,0.080701


In [28]:
df.to_csv(path_to_raw+'resultados_tfidf/tfidf.csv', sep=',', encoding='utf-8')       